In [2]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import  SIMPLE_MOVEMENT
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [3]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v2')
# 2. Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [4]:
import os
from  stable_baselines3 import PPO
from  stable_baselines3 import DQN
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import BaseCallback

In [5]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [6]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [7]:
#callback = TrainAndLoggingCallback(check_freq=10_000, save_path=CHECKPOINT_DIR)

In [13]:
model = PPO.load('250_000stepsMarioV2PPO')
model

In [14]:
model.set_env(env)
model.learn(total_timesteps=100_000)
model.save('350_000stepsMarioV2PPO')

Wrapping the env in a VecTransposeImage.
Logging to ./logs/PPO_4
----------------------------
| time/              |     |
|    fps             | 218 |
|    iterations      | 1   |
|    time_elapsed    | 2   |
|    total_timesteps | 512 |
----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 92          |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 1024        |
| train/                  |             |
|    approx_kl            | 0.006087376 |
|    clip_fraction        | 0.0156      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.4        |
|    explained_variance   | 0.644       |
|    learning_rate        | 1e-06       |
|    loss                 | 154         |
|    n_updates            | 2950        |
|    policy_gradient_loss | -0.000695   |
|    value_loss           | 440         |
------------------